# Analogous cycles
* This notebook implements the analogous cycles method on the experimentla visual data

In [ ]:
using Pkg
Pkg.activate("../../env/.")

In [1]:
include("../../src/analogous_bars.jl")
include("../../src/Eirene_var.jl")

using .analogous_bars
using .Eirene_var


WebIO._IJuliaInit()

┌ Warning: Error requiring `HDF5` from `Plots`
│   exception = (LoadError("/Users/irisyoon/.julia/packages/Plots/5S9Hg/src/backends/hdf5.jl", 36, UndefVarError(:Group)), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x000000010a00c2cf, Ptr{Nothing} @0x000000010a0a1206, Ptr{Nothing} @0x000000010a0a223b, Ptr{Nothing} @0x000000010a0a0b94, Ptr{Nothing} @0x000000010a0a0f0c, Base.InterpreterIP in top-level CodeInfo for Plots._hdf5_implementation at statement 4, Ptr{Nothing} @0x000000010a0bb844, Ptr{Nothing} @0x000000010a0baf65, Ptr{Nothing} @0x000000010a0bb604, Ptr{Nothing} @0x000000010a0bb604, Ptr{Nothing} @0x000000010a0bc134, Ptr{Nothing} @0x000000011f02d4e7, Ptr{Nothing} @0x000000010a08afbf, Ptr{Nothing} @0x000000011f07e312, Ptr{Nothing} @0x000000011ecb4a65, Ptr{Nothing} @0x000000010a08afbf, Ptr{Nothing} @0x000000010a097839, Ptr{Nothing} @0x000000010b66382e, Ptr{Nothing} @0x000000010a08afbf, Ptr{Nothing} @0x000000010a0a225f, Ptr{Nothing} @0x000000010a0a0b94, Ptr{Nothing} @0x000000

In [2]:
using DelimitedFiles
using HDF5
using Plots
using FileIO
using MultivariateStats


## 1. Load dissimialrity

In [3]:
# load dissimilarity
total_D = readdlm("total_dissimilarity.csv")

# get number of neurons in AL
AL_txt = open("AL_neurons_idx.txt", "r")
AL_linebyline = readlines(AL_txt)
close(AL_txt)
n_AL = size(AL_linebyline, 1)

# get number of neurons in V1
V1_txt = open("V1_neurons_idx.txt", "r")
V1_linebyline = readlines(V1_txt)
close(V1_txt)
n_V1 = size(V1_linebyline, 1)

# Define submatrices 
D_V1 = total_D[1:n_V1, 1:n_V1]
D_AL = total_D[n_V1+1:end, n_V1+1:end]
D_V1_AL = total_D[1:n_V1, n_V1+1:end]
    # rows (landmarks): V1
    # columns (witness) : AL
D_AL_V1 = total_D[n_V1+1:end, 1:n_V1];
    # rows (landmarks): AL
    # columns (witness) : V1

## 2. Run persistence

In [4]:
C_V1 = eirene(D_V1, record = "all", maxdim = 1)
C_AL = eirene(D_AL, record = "all", maxdim = 1)
W_V1 = ext_var.compute_Witness_persistence(D_V1_AL)

barcode_V1 = barcode(C_V1, dim = 1)
barcode_AL = barcode(C_AL, dim = 1)
barcode_W_V1_AL = barcode(W_V1["eirene_output"], dim = 1);

In [5]:
p1 = ext_var.plot_PD(barcode_V1, title = "PD(VR(V1))", titlefontsize = 15)
p2 = ext_var.plot_PD(barcode_W_V1_AL, title = "PD(W(V1, AL))")
p3 = ext_var.plot_PD(barcode_AL, title = "PD(VR(AL))", titlefontsize = 15)
plot(p1, p2, p3, layout = grid(1,3))

## 3. Compute similarity-centric analogous cycles

In [13]:
# select interval
W_class = 2
dim = 1;

In [14]:
extension_V1, extension_AL = run_similarity_analogous(
                            VR_P = C_V1, 
                            D_P = D_V1, 
                            VR_Q = C_AL,
                            D_Q = D_AL,
                            W_PQ = W_V1,
                            W_PQ_bar = W_class,
                            dim = dim);

In [15]:
plot_analogous_bars(extension_V1, extension_AL)

Note: In the above plot, `R1` refers to `V1` and `R2` refers to `AL`

In [48]:
M = fit(MDS, D_V1; maxoutdim=2)

Classical MDS(indim = 53, outdim = 2)

In [51]:

y = MultivariateStats.transform(M)

2×53 Matrix{Float64}:
 -0.287624  -1.24924   0.00472278  …  -1.06189   -1.43208   -0.369885
  0.663034  -0.367433  0.669586        0.128376  -0.654685   1.04739

In [58]:
y[1,10]

1.322305256433991

In [65]:
p = scatter(y[1,:], y[2,:])
scatter!(y[1, V1_cr_vertex], y[2, V1_cr_vertex])
for j = 1:size(V1_cr, 2)
    p1 = V1_cr[1,j]
    p2 = V1_cr[2,j]
    plot!([y[1, p1], y[1, p2]], [y[2, p1], y[2, p2]])
end
plot(p)

In [ ]:
V1_cr = classrep(C_V1, dim = 1, class = 2)

In [54]:
V1_cr_vertex = classrep(C_V1, dim = 1, class = 2, format = "vertex")

11-element Vector{Int64}:
 15
  8
 46
 52
 11
 20
 48
 50
  5
  7
 44

# Plot cycle representatives of the analogous pair 

In VR(P)

In [41]:
V1_cr = classrep(C_V1, dim = 1, class = 2)

2×11 Matrix{Int64}:
 11  44   8  48   5  52   7  48  52  44   5
  8   7  15  11  20  15  50  20  46  46  50

In [42]:
V1_cr_idx = zeros((size(V1_cr)))
for i = 1:size(V1_cr, 1)
    for j = 1:size(V1_cr, 2)
        V1_cr_idx[i,j] = parse(Int64, V1_linebyline[V1_cr[i,j]])
    end
end

In [43]:
n_V1

53

In [44]:
V1_cr_idx

2×11 Matrix{Float64}:
 88.0  309.0   65.0  330.0   43.0  341.0   45.0  330.0  341.0  309.0   43.0
 65.0   45.0  103.0   88.0  138.0  103.0  333.0  138.0  315.0  315.0  333.0

cycle rep: [88, 65, 103, 341, 315, 309, 45, 333, 43, 138, 330]

In VR(Q)

In [20]:
AL_cr = classrep(C_AL, dim = 1, class = 2)

2×5 Matrix{Int64}:
 32  32   7  24  21
 24   7  41  21  41

In [36]:
AL_cr_idx = zeros((size(AL_cr)))
for i = 1:size(AL_cr, 1)
    for j = 1:size(AL_cr, 2)
        AL_cr_idx[i,j] = parse(Int64, AL_linebyline[AL_cr[i,j]])
    end
end

In [37]:
AL_cr_idx

2×5 Matrix{Float64}:
 129.0  129.0   44.0  108.0   99.0
 108.0   44.0  161.0   99.0  161.0

cycle rep: [129, 108, 99, 161, 44]

## 4 Explore other possible extensions


### 4(a) Examine confirmed and possible extensions in extension_V1

<b> Plot confirmed interval and possible extensions </b>

In [35]:
plot_confirmed_extension(extension_V1, 
                         lw_x = 3, 
                         lw_y = 3, 
                         titlefontsize = 15, 
                         xtickfontsize = 12)

<!DOCTYPE html>
 
 
 Plots.jl

<b> Plot possible extensions </b>
* Plot the barcodes `barcode(VR(V1))`
* Use the slider to select parameter $\epsilon$. 
    * The dotted vertical line indicates the selected $\epsilon$.
* Select `intervals`. 
    * The pink intervals on the top barcode represented the confirmed intervals.
    * The blue interval(s) on the bottom barcode represent the selected possible interval.

In [36]:
plot_possible_extensions(extension_V1, lw = 4)

WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["ε"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 1,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
1, nothing)), Set{String}(), nothing, WebIO.Asset[WebIO.Asset("js", "knockout", "/Users/irisyoon/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), WebIO.Asset("js", "knockout_punches", "/Users/irisyoon/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), WebIO.Asset("js", nothing, "/Users/irisyoon/.julia/packages/InteractBase/Zfu5P/src/../assets/all.js"), WebIO.Asset("css", nothing, "/Users/irisyoon/.julia/packages/InteractBase/Zfu5P/src/../assets/style.css"), WebIO.Asset("css", nothing, "/Users/irisyoon/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{WebIO.AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000012ae2d210, Task (runnable) @0x000000012ae2d210), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.558992\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"8795968927446581932\",\"id\":\"10041344173540394904\",\"type\"

<b>Alternative representations under a different choice of basis of barcode(VR(V1))</b>

Since there are no other possible intervals at the selected parameter, there are no other alternative representations to consider.

### 4(b) Exaine confirmed and possible extensions in extension_AL

<b> Plot confirmed interval and possible extensions </b>

In [47]:
plot_confirmed_extension(extension_AL, 
                         lw_x = 3, 
                         lw_y = 3, 
                         titlefontsize = 15, 
                         xtickfontsize = 12)

<!DOCTYPE html>
 
 
 Plots.jl

<b> Plot possible extensions </b>
* Plot the barcodes `barcode(VR(AL))`
* Use the slider to select parameter $\epsilon$. 
    * The dotted vertical line indicates the selected $\epsilon$.
* Select `intervals`. 
    * The pink intervals on the top barcode represented the confirmed intervals.
    * The blue interval(s) on the bottom barcode represent the selected possible interval.

In [48]:
plot_possible_extensions(extension_AL, lw = 4)

WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["ε"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 2,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
1, nothing)), Set{String}(), nothing, WebIO.Asset[WebIO.Asset("js", "knockout", "/Users/irisyoon/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), WebIO.Asset("js", "knockout_punches", "/Users/irisyoon/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), WebIO.Asset("js", nothing, "/Users/irisyoon/.julia/packages/InteractBase/Zfu5P/src/../assets/all.js"), WebIO.Asset("css", nothing, "/Users/irisyoon/.julia/packages/InteractBase/Zfu5P/src/../assets/style.css"), WebIO.Asset("css", nothing, "/Users/irisyoon/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{WebIO.AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000012c142650, Task (runnable) @0x000000012c142650), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.456083\",\"0.50357\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"15178161629409657333\",\"id\":\"631699243384624861

<b>Alternative representations under a different choice of basis of barcode(VR(AL))</b>

Since there are no other possible intervals at the selected parameter, there are no other alternative representations to consider.